## torch._C 模块  

torch._C 模块是c++语言写的对pytorch扩展的库，pytorch很多类都继承自._C模块  
  
###   torch._ C的引用  
  
  在使用 
```python 
  import torch
```
的使用 ，按照python规范，会先执行 torch/__init__.py文件  
  
在torch/__init__.py文件
```python 
  import torch._C
```
引用torch 模块时，在torch的初始化模块里又引用了torch._ C模块  
  
<br>  
  
### torch._ C 模块源码  

以下都是 _ C的模块文件中源码。（模块名叫_ C ,文件名叫 Module.cpp）

创建_C模块结构体
```C++
   static struct PyModuleDef torchmodule = {
     PyModuleDef_HEAD_INIT,
     "torch._C",
     nullptr,
     -1,
     methods.data()  
  };
``` 
模块中含有的方法和变量的数组
```C++ 

static std::vector<PyMethodDef> methods;

```

初始化_C模块  
```C++ 

ASSERT_TRUE(module = PyModule_Create(&torchmodule));

```   
  
<br>  
  
$\color{red}{往\_C模块里添加成员和方法：}$ （往模块里添加方法是初始化里完成，往模块里添加成员，是在其他任意时候。）     
  
添加成员（调用要添加成员模块的初始化函数。这些成员模块的初始化函数的参数，是其父类的模型的引用）。
```C++  
 #例如往_ C模块里添加 _ TensorBase 模块。 
 # 调用 _ TensorBase 模块 的 初始化函数。并传入_ C模块的引用
 #  _ TensorBase的文件名是python_variabl.h 要在_C的文件里引用。
  ASSERT_TRUE(THPVariable_initModule(module));
```

<br>    
  
<br>   
  

<br>    
    

以下是_ C模块里被添加模块的文件。（被添加模块也要按照python对C扩展规范编写。） _ TensorBase的文件名是python_variabl.cpp  
  
模块名叫 _ TensorBase ， 文件名 python_variabl.cpp
  
```C++
# _ TensorBase 结构体 ，name 和 方法 数组  等一系列变量

PyTypeObject THPVariableType = {
  PyVarObject_HEAD_INIT(nullptr, 0)
  "torch._C._TensorBase",                      /* tp_name */
  
      ……
  nullptr,                                     /* tp_methods */
 
  
        ……    

};  


#_ TensorBase的初始化，参数是_C模块的引用
# vector<PyMethodDef> methods  把_ TensorBase的函数加入犯法数组  
# PyModule_AddObject(module, "_TensorBase",   (PyObject *)&THPVariableType);  把THPVariableType类型在_C里注册为_TensorBase  
#  第一个参数是父类，第二个参数是别名，第三个参数是要加入的类型。
# 同时这个初始化里还会为初始化initTorchFunctions，initTensorImplConversion这两个函数
bool THPVariable_initModule(PyObject *module)
{
  static std::vector<PyMethodDef> methods;
  THPUtils_addPyMethodDefs(methods, torch::autograd::variable_methods);
  THPUtils_addPyMethodDefs(methods, extra_methods);
  THPVariableType.tp_methods = methods.data();
  if (PyType_Ready(&THPVariableType) < 0)
    return false;
  Py_INCREF(&THPVariableType);
  PyModule_AddObject(module, "_TensorBase",   (PyObject *)&THPVariableType);
  torch::autograd::initTorchFunctions(module);
  torch::autograd::initTensorImplConversion(module);
  return true;
}

```